Inspired By This Discussion.
https://www.kaggle.com/c/riiid-test-answer-prediction/discussion/207148

In the original discussion the idea was to use conditional probability. My idea is to use association analysis. Then using it with KMeans clustering.


In [ ]:
!pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null 2>&1

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
import datatable as dt
import lightgbm as lgb
from matplotlib import pyplot as plt
import riiideducation
import random
from sklearn.metrics import roc_auc_score
import gc

In [ ]:
data_types_dict = {
    # 'timestamp': 'int64',
    'user_id': 'int32', 
    'content_id': 'int16', 
    # 'content_type_id':'int8', 
    # 'task_container_id': 'int16',
    #'user_answer': 'int8',
    'answered_correctly': 'int8', 
    # 'prior_question_elapsed_time': 'float32', 
    # 'prior_question_had_explanation': 'bool'
}
target = 'answered_correctly'

In [ ]:
train_df = dt.fread('../input/riiid-test-answer-prediction/train.csv', columns=set(data_types_dict.keys())).to_pandas()

In [ ]:
train_df = train_df.astype(data_types_dict)
train_df = train_df[train_df[target] != -1].reset_index(drop=True)

In [ ]:
sample = train_df[1000000:2000000]
sample

In [ ]:
limit = 1000
content_agg = train_df.groupby("content_id")[target].agg(["count"])
items = content_agg["count"].sort_values()[-limit * 2:]
items = pd.DataFrame(dict(index_value=items.index, count=items), index=items.index)
items = items.sort_values(["count", "index_value"])[-limit:][["count"]]
items

In [ ]:
sample = sample[sample.content_id.isin(items.index)]
user_table = sample.groupby(["user_id", "content_id"])[target].max()
user_table = user_table.unstack()
user_table = user_table.fillna(0).astype("int8")
user_table

In [ ]:
from mlxtend.frequent_patterns import apriori, fpmax, association_rules


freq_items1 = apriori(user_table, min_support=0.01, use_colnames=True, max_len=2)
freq_items1

In [ ]:
a_rules1 = association_rules(freq_items1, metric="confidence", min_threshold=0)
a_rules1 = a_rules1.sort_values('confidence', ascending=False).reset_index(drop=True)
a_rules1

In [ ]:
item_ids = items.index[::-1]
confidence_df = pd.DataFrame(np.zeros((len(item_ids), len(item_ids))), index=item_ids,
                            columns=item_ids)
confidence_df

In [ ]:
values = a_rules1.values
for i in range(len(values)):
    ante = list(values[i, 0])[0]
    conse = list(values[i, 1])[0]
    confidence = values[i, 5]
    # print(conse, ante, confidence)
    confidence_df.loc[conse, ante] = confidence
confidence_df

In [ ]:
%%time
from sklearn.cluster import KMeans


kmeans = KMeans(30, random_state=2)
kmeans.fit(confidence_df)

labels = pd.DataFrame(dict(content_id=confidence_df.index, label=kmeans.labels_), index=confidence_df.index)
labels

In [ ]:
labels.to_csv("labels.csv", index=False)

In [ ]:
count = labels.groupby("label")["label"].count()
plt.barh(count.index, width=count)

In [ ]:
confidence_df.loc[labels.label==20, labels[labels.label==20].index]